# DataJoint Element Facemap: Pose Estimation

Open-source data pipeline to automate analyses and organize data

In this tutorial, we will walk through processing facial behavior videos using the [Facemap Pose Estimation framework]("https://github.com/MouseLand/facemap"). Distinct facial keypoints are determined using a pose estimation model and are stored in the DataJoint pipeline. 

We will explain the following concepts as they relate to this pipeline:
- What is an Element versus a pipeline?
- Plot the pipeline with `dj.Diagram`
- Insert data into tables
- Query table contents
- Fetch table contents
- Run the pipeline for your experiments

For detailed documentation and tutorials on general DataJoint principles that support collaboration, automation, reproducibility, and visualizations:

- [DataJoint for Python - Interactive Tutorials](https://github.com/datajoint/datajoint-tutorials) - Fundamentals including table tiers, query operations, fetch operations, automated computations with the `make` function, etc.

- [DataJoint for Python - Documentation](https://datajoint.com/docs/core/datajoint-python/)

- [DataJoint Element for Facemap - Documentation](https://datajoint.com/docs/elements/element-facemap/)

Let's start by importing the packages necessary to run this pipeline 


# DataJoint Element for Facemap Deep Learning

#### Open-source data pipeline for processing and analyzing facial behavior videos.

Welcome to the tutorial for the DataJoint Element for facial pose estimation and ROI detection. This
tutorial aims to provide a comprehensive understanding of the open-source data pipeline
created using `element-facemap`.

This package is designed to seamlessly process, ingest, and track extracellular electrophysiology
data, along with its associated probe and recording metadata. By the end of this
tutorial you will have a clear grasp on setting up and integrating `element-facemap`
into your specific research projects and lab. 

![flowchart](../images/diagram_flowchart.svg)

### Prerequisites

Please see the [datajoint tutorials GitHub
repository](https://github.com/datajoint/datajoint-tutorials/tree/main) before
proceeding.

A basic understanding of the following DataJoint concepts will be beneficial to your
understanding of this tutorial: 
1. The `Imported` and `Computed` tables types in `datajoint-python`.
2. The functionality of the `.populate()` method. 

#### **Tutorial Overview**

+ Setup
+ *Activate* the DataJoint pipeline.
+ *Insert* subject and session metadata.
+ *Populate* Video Recordings and Model information
+ Generate and run the pose estimation task.
+ Visualize the results.

### **Setup**

This tutorial examines video data of .mp4 file format. The goal is to store, track
and manage sessions of facial pose data, including determining coordinates of facial body parts and
trajectory visualizations.

The results of this Element can be combined with **other modalities** to create
a complete, customizable data pipeline for your specific lab or study. For instance, you
can combine `element-facemap` with `element-calcium-imaging` and
`element-array-ephys` to relate orofacial behavior to the neural activity.

Let's start this tutorial by importing the packages necessary to run the notebook.

In [ ]:
import datajoint as dj
import datetime
import os
import matplotlib.pyplot as plt

If the tutorial is run in Codespaces, a private, local database server is created and
made available for you. This is where we will insert and store our processed results.
Let's connect to the database server.

In [ ]:
dj.conn()

### **Activate the DataJoint Pipeline**

This tutorial activates the `facemap_inference.py` module from `element-facemap`, along
with upstream dependencies from `element-animal` and `element-session`. Please refer to the
[`tutorial_pipeline.py`](./tutorial_pipeline.py) for the source code.

## Combine multiple Elements into a pipeline

Each DataJoint Element is a modular set of tables that can be combined into a complete pipeline.

Each Element contains 1 or more modules, and each module declares its own schema in the database.

This tutorial pipeline is assembled from four DataJoint Elements.

The results of this Element example can be combined with other modalities to create a complete customizable data pipeline for your specific lab or study. For instance, you can combine element-facemap with element-calcium-imaging and element-array-ephys to characterize the neural activity.

| Element | Source Code | Documentation | Description |
| -- | -- | -- | -- |
| Element Lab | [Link](https://github.com/datajoint/element-lab) | [Link](https://datajoint.com/docs/elements/element-lab) | Lab management related information, such as Lab, User, Project, Protocol, Source. |
| Element Animal | [Link](https://github.com/datajoint/element-animal) | [Link](https://datajoint.com/docs/elements/element-animal) | General animal metadata and surgery information. |
| Element Session | [Link](https://github.com/datajoint/element-session) | [Link](https://datajoint.com/docs/elements/element-session) | General information of experimental sessions. |
| Element Facemap | [Link](https://github.com/datajoint/element-facemap) | [Link](https://datajoint.com/docs/elements/element-facemap) |  Facemap orofacial bodypart prediction with singular value decomposition or deep learning pose estimation.|

By importing the modules for the first time, the schemas and tables will be created in the database.  Once created, importing modules will not create schemas and tables again, but the existing schemas/tables can be accessed.

The Elements are imported and activated within the `tutorial_pipeline` script.

In [ ]:
from tutorial_pipeline import (
    lab,
    subject,
    session,
    fbe,
    facemap_inference,
    Device
)

Each Python module (e.g. subject) contains a schema object that enables interaction with the schema in the database.

In [ ]:
subject.schema

The Python classes in the module correspond to a table in the database server.

In [ ]:
subject.Subject()

## Facemap Inference Schemas Diagram

We can represent the `fbe` and `facemap_inference` schemas and their upstream dependencies, `subject` and `session`, using `dj.Diagram()`.

In [ ]:
(
    dj.Diagram(subject.Subject)
    + dj.Diagram(session.Session)
    + dj.Diagram(fbe.VideoRecording)
    + dj.Diagram(facemap_inference)
)

As evident from the diagram, this data pipeline encompasses tables associated with
model and video file data, task generation and results of model inference. A few tables, such as `subject.Subject` or `session.Session`,
while important for a complete pipeline, fall outside the scope of the `element-facemap`
tutorial, and will therefore, not be explored extensively here. The primary focus of
this tutorial will be on the `facemap_inference` schema.

## Insert entries into `subject` and `session` manual tables

Let's start with the first table in the schema diagram (i.e. `subject.Subject` table).

To know what data to insert into the table, we can view its dependencies and attributes using the `.describe()` and `.heading` functions.

In [ ]:
print(subject.Subject.describe())

In [ ]:
subject.Subject.heading

The cells above show all attributes of the subject table.
We will insert data into the
`subject.Subject` table. 

In [ ]:
subject.Subject.insert1(
    dict(
        subject="subject1",
        subject_nickname="subject1_nickname",
        sex="U",
        subject_birth_date="2020-01-01",
        subject_description="Demo subject for Facemap Pose estimation processing.",
    )
)
subject.Subject()

Let's repeat the steps above for the `Session` table.

In [ ]:
print(session.Session.describe())

In [ ]:
session.Session.heading

Notice that `describe`, displays the table's structure and highlights its dependencies, such as its reliance on the `Subject` table. These dependencies represent foreign key references, linking data across tables.

On the other hand, `heading` provides an exhaustive list of the table's attributes. This
list includes both the attributes declared in this table and any inherited from upstream
tables.

With this understanding, let's move on to insert a session associated with our subject.

We will insert into the `session.Session` table by passing a dictionary to the `insert1` method.

In [ ]:
session_key = dict(subject="subject1", session_datetime="2021-04-30 12:22:15")

In [ ]:
session.Session.insert1(session_key)
session.Session()

Every experimental session produces a set of data files. The purpose of the `SessionDirectory` table is to locate these files. It references a directory path relative to a root directory, defined in `dj.config[\"custom\"]`. More information about `dj.config` is provided in the [documentation](https://datajoint.com/docs/elements/user-guide/).

In [ ]:
session.SessionDirectory.insert1(
    dict(**session_key, session_dir="subject1/session1")
)
session.SessionDirectory()

With the instantiation of the element-facemap

Let's start by inserting a local pytorch model file into the `facemap_inference.FacemapModel` table
- Specify a unique `model_id`, a `model_description`, and the `full_local_model_filepath`
- The default facemap model is located in the hidden .facemap folder installed to your computer's home directory: `i.e. ~/.facemap/models/facemap_model_state.pt` 

In [ ]:
model_name = 'facemap_model_state.pt'
model_id = 0
model_description = "test facemap model"
full_local_model_filepath = "~/.facemap/models/facemap_model_state.pt"
facemap_inference.FacemapModel.insert_new_model(
                                                model_name=model_name, 
                                                model_id=model_id, 
                                                model_description=model_description, 
                                                full_model_path=full_local_model_filepath
                                                )

Let's display the `facemap_inference.FacemapModel` table queried with the model id of interest to verify insertion

In [ ]:
facemap_inference.FacemapModel() & f'model_id={model_id}'

Next let's display the `facemap_inference.FacemapModel.File` and `facemap_inference.FacemapModel.BodyPart` part tables

In [ ]:
facemap_inference.FacemapModel.File() & f'model_id={model_id}'

In [ ]:
facemap_inference.FacemapModel.BodyPart() & f'model_id={model_id}'

As the Diagram indicates, `fbe.VideoRecording` table needs to
contain data before the `facemap_inference.FacemapPoseEstimationTask` can be generated.

Next we will insert behavioral video recording data into `fbe.VideoRecording` and its part table, `fbe.VideoRecording.File`. 

In [ ]:
from pathlib import Path
video_recording_insert = {**session_key, 
                          "recording_id": 0}
facemap_root_dir_path = fbe.get_facemap_root_data_dir()
vid_path = "example_behavior_videos mounted S3 inbox path"
video_recording_file_insert = {
    **video_recording_insert,
    "file_id": 0,
    "file_path": Path(vid_path).relative_to(facemap_root_dir_path),
}

fbe.VideoRecording.insert1(video_recording_insert)
fbe.VideoRecording.File.insert1(video_recording_file_insert)

With an entries present in the `facemap_inference.FacemapModel` and the `fbe.VideoRecording` tables, the criteria is met for insertion into the `facemap_inference.FacemapPoseEstimationTask` table.
- `facemap_inference.FacemapPoseEstimationTask` is a staging table that pairs a specific `FacemapModel` with a specific `VideoRecording`.

 Pose estimation is then be evaluated in the next table `facemap_inference.FacemapPoseEstimation` according the the specifications of the `FacemapPoseEstimationTask`

## Run Facemap Inference Pipeline

In [ ]:
def ingest_model(model_name, model_description, full_model_path):
    try:
        model_id = max(facemap_inference.FacemapModel.fetch("model_id"))
        model_id = model_id + 1  # increment model id
    except ValueError:  # case that nothing has been inserted
        model_id = 0

    facemap_model_insert = dict(
        model_id=model_id, model_name=model_name, model_description=model_description
    )
    facemap_inference.FacemapModel.insert1(facemap_model_insert)

    body_part_insert = []
    body_parts = [
        "eye(back)",
        "eye(bottom)",
        "eye(front)",
        "eye(top)",
        "lowerlip",
        "mouth",
        "nose(bottom)",
        "nose(r)",
        "nose(tip)",
        "nose(top)",
        "nosebridge",
        "paw",
        "whisker(I)",
        "whisker(III)",
        "whisker(II)",
    ]
    for bp in body_parts:
        body_part_insert.append(dict(model_id=model_id, body_part=bp))
    # Insert into parent BodyPart table if no entries are present
    if len(facemap_inference.BodyPart()) == 0:
        facemap_inference.BodyPart.insert(body_part_insert)
    file_insert = dict(model_id=model_id, model_file=full_model_path)

    facemap_inference.FacemapModel.BodyPart.insert(body_part_insert)
    facemap_inference.FacemapModel.File.insert1(file_insert)

#### Insert Subject and Session into subject.Subject, session.Session and session.SessionDirectory tables

In [ ]:
sub_insert = dict(subject="mdl_sub", 
                subject_nickname="facemap model subject", 
                sex='U', 
                subject_birth_date=datetime.datetime.now(), 
                subject_description="Subject for Facemap Model Inference testing")
# subject.Subject.insert1(sub_insert)
subject_key = (subject.Subject & 'subject="mdl_sub"').fetch1("KEY")

In [ ]:
session_id = 2
session_insert = dict(subject_key, session_id, session_datetime=datetime.datetime.now())
sdir_insert = dict(subject_key, session_id, session_dir="20230627_Image_eCBsensor_activity/Behavior_20230627/C57-C11-3_Rm_CNO_30min")

session.Session.insert1(session_insert)
session.SessionDirectory.insert1(sdir_insert)

In [ ]:
# Display Session Table to validate insert
session.Session() & {**subject_key, 'session_id': session_id}

In [ ]:
# Display SessionDirectory Table to validate insert
session.SessionDirectory() & {**subject_key, 'session_id': session_id}

### Ingest locally stored pytorch model(.pt) file
Provide model name, model filepath, and optional model description

In [ ]:
model_name = 'facemap_model_state.pt'
full_local_model_filepath = "/Users/sidhulyalkar/.facemap/models/facemap_model_state.pt"
ingest_model(model_name, model_description="test facemap model", model_file=full_local_model_filepath)

### Generate a Pose Estimation Task

In [ ]:
model_id = 0
session_key = session.Session.fetch("KEY")[2] 
generate_facemap_inference_estimation_task(model_id, session_key, task_mode="trigger", bbox=[])

#### Display FacemapPoseEstimationTask table

In [ ]:
facemap_inference.FacemapPoseEstimationTask()

#### Display VideoRecording and VideoRecording.File tables from the imported facial behavioral estimation (fbe) schema

In [ ]:
fbe.VideoRecording()

In [ ]:
fbe.VideoRecording.File()

#### Run Pose Estimation on all unprocessed FacemapPoseEstimationTasks  

In [ ]:
facemap_inference.FacemapPoseEstimation.populate(display_progress=True)
# If a lost connection error occurs, rerun the populate and if processing 
# has completed, the data will be loaded and inference will not be rerun. 


#### Display Facemap Pose Estimation Tables

In [ ]:
facemap_inference.FacemapPoseEstimation()

In [ ]:
facemap_inference.FacemapPoseEstimation.BodyPartPosition()

#### Visualize Pose Estimation Output

In [ ]:
pe_query = {**session_key, 'recording_id': 0, 'model_id': model_id}
pose_estimation_key = (facemap_inference.FacemapPoseEstimation & pe_query).fetch1("KEY")

Get Trajectory of X and Y coordinates

In [ ]:
# Specify all body parts, or set body_parts to a custom list
body_parts = "all"
model_name = (facemap_inference.FacemapModel & f'model_id={key["model_id"]}').fetch1("model_name")

if body_parts == "all":
    body_parts = (facemap_inference.BodyPartPosition & key).fetch("body_part")
elif not isinstance(body_parts, list):
    body_parts = list(body_parts)


In [ ]:
# Construct Pandas MultiIndex DataFrame
df = None
for body_part in body_parts:
    result_dict = (
        facemap_inference.BodyPartPosition
        & {"body_part": body_part}
        & {"recording_id": key["recording_id"]}
        & {"session_id": key["session_id"]}
    ).fetch("x_pos", "y_pos", "likelihood", as_dict=True)[0]
    x_pos = result_dict["x_pos"].tolist()
    y_pos = result_dict["y_pos"].tolist()
    likelihood = result_dict["likelihood"].tolist()
    a = np.vstack((x_pos, y_pos, likelihood))
    a = a.T
    pdindex = pd.MultiIndex.from_product(
        [[model_name], [body_part], ["x", "y", "likelihood"]],
        names=["model", "bodyparts", "coords"],
    )
    frame = pd.DataFrame(a, columns=pdindex, index=range(0, a.shape[0]))
    df = pd.concat([df, frame], axis=1)
df

In [ ]:
# Or can use the built in function get_trajectory which also constructs this Pandas MultiIndex DataFrame
# df=facemap_inference.FacemapPoseEstimation.get_trajectory(pose_estimation_key)
# df

In [ ]:
df_xy = df.iloc[:,df.columns.get_level_values(2).isin(["x","y"])]['facemap_model_state.pt']
df_xy.mean()

Plot coordinates across time for each body part

In [ ]:
df_xy.plot().legend(loc='best', prop={'size': 5})

In [ ]:
df_flat = df_xy.copy()
df_flat.columns = df_flat.columns.map('_'.join)

Plot Trace Overlays of each body part across time

In [ ]:
fig,ax=plt.subplots(2,2)
fig.set_figwidth(20)
fig.set_figheight(15)

df_flat.plot(x='eye(front)_x',y='eye(front)_y',ax=ax[0, 0])
df_flat.plot(x='eye(back)_x',y='eye(back)_y',ax=ax[0, 0])
df_flat.plot(x='eye(bottom)_x',y='eye(bottom)_y',ax=ax[0, 0])

df_flat.plot(x='nose(tip)_x',y='nose(tip)_y', ax=ax[1, 0])
df_flat.plot(x='nose(bottom)_x',y='nose(bottom)_y', ax=ax[1, 0])
df_flat.plot(x='nose(r)_x',y='nose(r)_y', ax=ax[1, 0])
df_flat.plot(x='nosebridge_x',y='nosebridge_y', ax=ax[1, 0])

df_flat.plot(x='mouth_x',y='mouth_y', ax=ax[0, 1])
df_flat.plot(x='lowerlip_x',y='lowerlip_y', ax=ax[0, 1])
df_flat.plot(x='paw_x',y='paw_y', ax=ax[0, 1])

df_flat.plot(x='whisker(I)_x',y='whisker(I)_y', ax=ax[1, 1])
df_flat.plot(x='whisker(II)_x',y='whisker(II)_y', ax=ax[1, 1])
df_flat.plot(x='whisker(II)_x',y='whisker(II)_y', ax=ax[1, 1])
